In [1]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [2]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  7325
Number of total ingredients:  545617


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [3]:
#caught a rogue ingredient
ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
ings = ings.drop_duplicates(subset = 'ingredient')
print('Number of unique ingredients: ', len(ings))
ings.head()

Number of unique ingredients:  7324


,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,Sesamum Indicum (Sesame) Seed Oil,1,A,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
ing_uniqueID = ings.loc[:,['ingredient']].reset_index()
ing_uniqueID = ing_uniqueID.drop(['index'], axis = 1)
ing_uniqueID['uniqueID'] = ing_uniqueID.index
#ing_uniqueID

### Prods
Prods df is a database of all the products with various information including brand, price and rating

In [5]:
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


### Prod_ing
Prod ing is a database that essentiall links prods and ings. it contains the product id, ingredient and ingredient order.

In [6]:
prod_ing.head(3)

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3


### Add a column that is a unique ingredient identifier

In [7]:
#id is product id
#uniqueID is ingredient ID
prod_ing = pd.merge(prod_ing, ing_uniqueID, on='ingredient')
prod_ing = prod_ing.sort_values(['id', 'order'])
prod_ing.head()

,id,ingredient,order,uniqueID
0,0,Water,1,0
15954,0,Hydrogenated Polyisobutene,2,1
16617,0,Butylene Glycol,3,2
24563,0,Sesamum Indicum (Sesame) Seed Oil,4,3
24889,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,5,4


In [8]:
#reset index and drop
prod_ing = prod_ing.reset_index(drop=True)

## Reformatting Prod_ing
Instead of having each ingredient in it's own row, we want to create a list of ingredients for each product while retaining the order.

In [9]:
prod_ing.groupby('id')['ingredient'].apply(list)

id
0        [Water, Hydrogenated Polyisobutene, Butylene G...
1        [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2        [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3        [Water, Glycereth-26, Alcohol, Butylene Glycol...
4        [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
5        [Water, Butylene Glycol, Glycerin, Disodium Su...
6        [Mineral Oil, Avena Sativa (Oat) Kernel Flour,...
7        [Sodium Palm Kernelate, Water, Sodium Borate, ...
8        [Water, Hamamelis Virginiana (Witch Hazel) Wat...
9        [Water, Caprylic/Capric Triglyceride, Glycerin...
10       [Water, Glycerin, Sodium Laureth Sulfate, PEG-...
11       [Water, Tri-C14-15 Alkyl Citrate, Cyclohexasil...
12       [Water, Butylene Glycol, Propylene Glycol, Dip...
13       [Sodium Palm Kernelate, Water, Petrolatum, Gly...
14       [Water, Chamaecyparis Obtusa Water, Glycerin, ...
15       [Snail Secretion Filtrate, Water, Butylene Gly...
16       [Water, Bentonite, Glycerin, Magnesium Alumi

In [10]:
#cleanup of a rogue ingredient here as well
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')

##group and create list
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_ID_lists = prod_ing.groupby('id')['uniqueID'].apply(list)

##convert back to dataframe and reset index
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_ID_df = prod_ing_ID_lists.to_frame().reset_index()
##check to make sure unique id's are still in tact (id shouldn't be exactly == to index)
prod_ing_lists.head()

id
0    [Water, Hydrogenated Polyisobutene, Butylene G...
1    [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2    [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3    [Water, Glycereth-26, Alcohol, Butylene Glycol...
4    [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
Name: ingredient, dtype: object

In [11]:
prod_ing_df.tail()

,id,ingredient
19858,19922,"[Water, Dimethicone, Glycerin, Palmitic Acid(E..."
19859,19923,"[Water, Dipropylene Glycol, Caprylic/Capric Tr..."
19860,19924,"[Water, Polyvinyl Alcohol, Ethanolamine, 1,2-H..."
19861,19925,"[Ricinus Communis (Castor) Seed Oil, Lanolin, ..."
19862,19926,"[Water, Ascorbic Acid (Vitamin C, Glycerin, Di..."


In [12]:
prod_ing_lists = pd.merge(prod_ing_df, prod_ing_ID_df, on='id')
prod_ing_lists = prod_ing_lists.rename(columns = {'ingredient': 'ingList', 'uniqueID': 'ing#List' })
prod_ing_lists.head()

,id,ingList,ing#List
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3..."
2,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55..."
3,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,..."
4,4,"[Petrolatum, Glycerin, Cocamidopropyl Betaine,...","[47, 23, 92, 93, 94, 2, 95, 96, 97, 98, 99, 16..."


## Merge prod_ings and prods
We can now use the list to merge back with the products.

In [13]:
products_and_ingredients= pd.merge(prod_ing_lists, prods, on='id')
products_and_ingredients.tail(3)
len(products_and_ingredients)

15945

### Add column for number of ingredients

In [14]:
products_and_ingredients['ingCount'] = products_and_ingredients['ingList'].apply(lambda x: len(x))
products_and_ingredients.head(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,...,0,0,0,1,0,0,0,1,0,22
1,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,...,0,1,0,1,0,0,1,1,0,20
2,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,...,0,1,1,0,0,1,1,1,0,38


In [15]:
type(products_and_ingredients.loc[0]['ing#List'])

list

In [16]:
#products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].apply(lambda x: str(x))
#products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(',', '')
#products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace('[', '')
#products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(']', '')
#products_and_ingredients = products_and_ingredients.drop_duplicates(subset = 'product')

len(products_and_ingredients)


15945

In [17]:
products_and_ingredients.to_csv('recommendations/recommender_products.csv')

# GET RECOMMENDATIONS


In [18]:
df = products_and_ingredients
#remove water
df['ing#List'] = [list(filter(lambda a: a != 0, i)) for i in df['ing#List']]

In [27]:
product ='Hyaluronic Acid 2% + B5'
ingTest = df[df['product'].str.match(product)]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

## Cosine similarity

In [37]:
%%time

recommended = get_COS_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/COS.csv')

Wall time: 26.7 s


In [38]:
topTen

,product,ingList,commonIng
11290,Booster Low-Molecular HA,"[Water, Hydrolyzed Yeast Extract, Pentylene Gl...",15
1479,Alpha Arbutin 2% + HA,"[Water, Alpha-Arbutin, Polyacrylate Crosspolym...",9
211,Matrixyl 10% + HA,"[Water, Glycerin, Butylene Glycol, Palmitoyl T...",10
11289,Booster Sensitive Fix,"[Water, Sodium Hyaluronate Crosspolymer, Sacch...",13
1888,SubQ Anti Age Advanced Serum,"[Water, Glycerin, Hydrolyzed Yeast Extract, Pe...",15
6646,Hydraluron Moisture Serum,"[Water, Propanediol, Sodium Hyaluronate, Carbo...",5
11022,Inhibitif Face Serum,"[Propanediol, Water, Rosa Damascena Flower Wat...",13
8306,Skin Freshener,"[Water, Pentylene Glycol, Saccharide Isomerate...",10
15478,Lactic Acid 5% + HA 2%,"[Water, Lactic Acid, Glycerin, Pentylene Glyco...",9
15514,Lactic Acid 10% + HA 2%,"[Water, Lactic Acid, Glycerin, Pentylene Glyco...",9


## Average Overlap

In [30]:
%%time

recommended = get_A0_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/AO.csv')

Wall time: 38.7 s


In [31]:
topTen

,product,ingList,commonIng
1583,Pure Vitamin C Serum,"[Sodium Hyaluronate, Ascorbic Acid (Vitamin C,...",1
10729,Concentrated Firming Serum,"[Water, Sodium Hyaluronate, Vegetable Glycerin...",2
4273,Youth Eye Complex,"[Water, Sodium Hyaluronate, Caprylic/Capric Tr...",5
12634,Sleeping Pack,"[Water, Sodium Hyaluronate, Ascorbyl Tetraisop...",6
12442,Hyaluronic Lift Serum W/ MDI Complex,"[Water, Sodium Hyaluronate, Panthenol, Polysor...",4
1694,Triple Age Repair Moisturizer with SPF 25,"[Benzophenone-2, Homosalate, Octocrylene, Wate...",4
12159,AmPm Super Triple HA Serum 30ml,"[Water, Sodium Hyaluronate, Glycosphingolipids...",5
8519,Hyaluron Serum,"[Water, Sodium Hyaluronate, Citric Acid]",3
703,Super Volcanic Pore Clay Mask,"[Water, Titanium Dioxide, Butylene Glycol, Vol...",4
3361,C-ESTA Serum,"[Water, Sodium Hyaluronate, Ascorbyl Palmitate...",3


## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [32]:
%%time

recommended = get_RBO_recommendations(df,product,rbo_min)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/RBO.csv')

Wall time: 48.5 s


In [33]:
topTen

,product,ingList,commonIng
1583,Pure Vitamin C Serum,"[Sodium Hyaluronate, Ascorbic Acid (Vitamin C,...",1
10729,Concentrated Firming Serum,"[Water, Sodium Hyaluronate, Vegetable Glycerin...",2
12634,Sleeping Pack,"[Water, Sodium Hyaluronate, Ascorbyl Tetraisop...",6
4273,Youth Eye Complex,"[Water, Sodium Hyaluronate, Caprylic/Capric Tr...",5
12442,Hyaluronic Lift Serum W/ MDI Complex,"[Water, Sodium Hyaluronate, Panthenol, Polysor...",4
703,Super Volcanic Pore Clay Mask,"[Water, Titanium Dioxide, Butylene Glycol, Vol...",4
12159,AmPm Super Triple HA Serum 30ml,"[Water, Sodium Hyaluronate, Glycosphingolipids...",5
3361,C-ESTA Serum,"[Water, Sodium Hyaluronate, Ascorbyl Palmitate...",3
1694,Triple Age Repair Moisturizer with SPF 25,"[Benzophenone-2, Homosalate, Octocrylene, Wate...",4
5333,Wrinkle Revenge Ultimate Hyaluronic Serum,"[Water, Sodium Hyaluronate Crosspolymer, Glyce...",9
